# MNA Multi Agent Analysis Code

## Libraries Installation

In [1]:
%pip install llama-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 8.4 MB/s eta 0:00:00


In [2]:
%pip install tavily-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 2.3 MB/s eta 0:00:00


In [3]:
!pip install llama-index-utils-workflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.3 MB/s eta 0:00:00


## Importing Libraries and API key setup

In [4]:
from google.colab import userdata
api_key = userdata.get('OPEN_AI_API_KEY')

With our API key in hand, instantiating an LLM is one line:

In [5]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4o-mini", api_key=api_key)

With our API key in hand, instantiating an LLM is one line:

A core feature of agents is that they can use tools to find out more about the state of the world, or take action in response to instructions, without you needing to be explicit about how and when they do that.

In this example, we'll give our agent the ability to search the web for information by creating a tool that does that. We'll use a service called Tavily, which is specifically designed to provide this kind of tool to agents. You can get a free API key from [Tavily](https://tavily.com/).

In [6]:
from google.colab import userdata
tavily_api_key = userdata.get('TAVILY_API_KEY')

Tools in LlamaIndex are just regular Python functions, so they can do anything a regular function can.

When creating a tool, its very important to:
- give the tool a distinctive name, and a clear description using docstrings. The LLM uses the name and description to understand what the tool does.
- annotate the types. This helps the LLM understand the expected input and output types.
- use async when possible, since this will make the workflow more efficient.

In [7]:
from llama_index.core.workflow import (
    Context,
    InputRequiredEvent,
    HumanResponseEvent,
)
from llama_index.core.agent.workflow import FunctionAgent, ReActAgent
from llama_index.core.agent.workflow import AgentWorkflow

from llama_index.core.agent.workflow import (
    AgentInput,
    AgentOutput,
    ToolCall,
    ToolCallResult,
    AgentStream,
)
from tavily import AsyncTavilyClient

from llama_index.core.agent import FunctionCallingAgent
from llama_index.core.tools import FunctionTool
from llama_index.core.workflow import Event
import yfinance as yf

from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
)
from llama_index.core.agent import FunctionCallingAgent
from llama_index.core.tools import FunctionTool

## Main Code

In [8]:

# note the type annotations for the incoming query and the return string
async def search_web(query: str) -> str:
    """Useful for using the web to answer questions."""
    client = AsyncTavilyClient(api_key=tavily_api_key)
    return str(await client.search(query))
# We only have access to the search_web tool
search_web_tool = FunctionTool.from_defaults(fn=search_web)

# Define system prompts for specialized agents that leverage web search
target_identification_agent = FunctionCallingAgent.from_tools(
    tools=[search_web_tool],
    llm=llm,
    verbose=False,
    system_prompt="""You are an agent that identifies and evaluates potential acquisition targets.
    Search for companies in the specified industry that match the given criteria.
    Return a ranked list of 3-5 suitable companies with brief justifications for each selection."""
)

financial_analysis_agent = FunctionCallingAgent.from_tools(
    tools=[search_web_tool],
    llm=llm,
    verbose=False,
    system_prompt="""You are an agent that analyzes financial data for companies.
    Search for and analyze financial information for both the target and acquirer companies.
    Assess financial compatibility, health, risks, and benefits of the acquisition.
    Format your response as a structured financial analysis."""
)

strategic_fit_agent = FunctionCallingAgent.from_tools(
    tools=[search_web_tool],
    llm=llm,
    verbose=False,
    system_prompt="""You are an agent that evaluates strategic fit between companies.
    Search for information about both companies and evaluate:
    1. Strategic alignment
    2. Product portfolio fit
    3. Market synergies
    4. Operational synergies
    5. Cultural compatibility
    Format your response as a structured strategic fit evaluation."""
)

legal_risk_agent = FunctionCallingAgent.from_tools(
    tools=[search_web_tool],
    llm=llm,
    verbose=False,
    system_prompt="""You are an agent that assesses legal and regulatory risks of acquisitions.
    Search for information and assess:
    1. Antitrust concerns
    2. Regulatory approval requirements
    3. Legal liabilities
    4. Compliance issues
    5. Jurisdictional challenges
    Format your response as a structured legal risk assessment."""
)

sentiment_agent = FunctionCallingAgent.from_tools(
    tools=[search_web_tool],
    llm=llm,
    verbose=False,
    system_prompt="""You are an agent that analyzes market sentiment regarding potential acquisitions.
    Search for and analyze:
    1. Market sentiment
    2. Media coverage
    3. Analyst opinions
    4. Stakeholder sentiments
    Format your response as a structured sentiment analysis."""
)

synergy_agent = FunctionCallingAgent.from_tools(
    tools=[search_web_tool],
    llm=llm,
    verbose=False,
    system_prompt="""You are an agent that estimates synergies from acquisitions.
    Based on available information, estimate:
    1. Revenue synergies
    2. Cost synergies
    3. Financial synergies
    4. Strategic synergies
    Include potential values and implementation timeline.
    Format your response as a structured synergy estimation."""
)

deal_structure_agent = FunctionCallingAgent.from_tools(
    tools=[search_web_tool],
    llm=llm,
    verbose=False,
    system_prompt="""You are an agent that recommends deal structures for acquisitions.
    Based on available information, recommend:
    1. Optimal transaction type
    2. Consideration structure
    3. Governance arrangements
    4. Key terms
    5. Timing considerations
    Format your response as a structured deal structure recommendation."""
)

valuation_agent = FunctionCallingAgent.from_tools(
    tools=[search_web_tool],
    llm=llm,
    verbose=False,
    system_prompt="""You are an agent that performs valuation for acquisitions.
    Based on available information, provide:
    1. Multiple valuation methods
    2. Premium analysis
    3. Comparable transactions
    4. Recommended price range
    Format your response as a structured valuation analysis."""
)

report_agent = FunctionCallingAgent.from_tools(
    tools=[],  # Report generation doesn't need web search
    llm=llm,
    verbose=False,
    system_prompt="""You are an agent that compiles comprehensive M&A analysis reports.
    Based on all the analyses provided, create a detailed report with:
    1. Executive summary
    2. Key findings from all analyses
    3. SWOT analysis
    4. Recommendations
    5. Next steps
    Format your response as a professional M&A analysis report."""
)

review_agent = FunctionCallingAgent.from_tools(
    tools=[],  # Report review doesn't need web search
    llm=llm,
    verbose=False,
    system_prompt="""You are an agent that reviews M&A analysis reports.
    Create a weighted decision matrix that evaluates key factors such as:
    1. Strategic value (25%)
    2. Financial impact (25%)
    3. Implementation feasibility (20%)
    4. Risk factors (20%)
    5. Timeline considerations (10%)
    Provide a final recommendation based on your review.
    Format your response as a decision matrix with scoring and final recommendation."""
)

In [9]:
# Define events for each step in the M&A analysis workflow
class TargetIdentificationEvent(Event):
    criteria: str
    industry: str
    acquirer_company: str
    target_company: str

class FinancialAnalysisEvent(Event):
    target_company: str
    acquirer_company: str

class StrategicFitEvent(Event):
    financial_analysis: str
    target_company: str
    acquirer_company: str

class LegalRegulatoryEvent(Event):
    strategic_fit: str
    target_company: str
    acquirer_company: str

class SentimentEvent(Event):
    legal_analysis: str
    target_company: str
    acquirer_company: str

class SynergyEvent(Event):
    sentiment_analysis: str
    target_company: str
    acquirer_company: str

class DealStructuringEvent(Event):
    synergy_estimation: str
    target_company: str
    acquirer_company: str

class ValuationEvent(Event):
    deal_structure: str
    target_company: str
    acquirer_company: str

class ReportGenerationEvent(Event):
    valuation: str
    target_company: str
    acquirer_company: str
    all_analysis: dict  # Contains all previous analyses

class ReportReviewEvent(Event):
    report: str

class ReviewResults(Event):
    review: str

class MnAAnalysisWorkflow(Workflow):
    @step
    async def setup(self, ev: StartEvent) -> TargetIdentificationEvent:
        # Initialize all agents
        self.target_id_agent = ev.target_id_agent
        self.financial_agent = ev.financial_agent
        self.strategic_agent = ev.strategic_agent
        self.legal_agent = ev.legal_agent
        self.sentiment_agent = ev.sentiment_agent
        self.synergy_agent = ev.synergy_agent
        self.deal_agent = ev.deal_agent
        self.valuation_agent = ev.valuation_agent
        self.report_agent = ev.report_agent
        self.review_agent = ev.review_agent

        # Dictionary to store all analysis results
        self.all_analysis = {}

        return TargetIdentificationEvent(
            criteria=ev.criteria,
            industry=ev.industry,
            acquirer_company=ev.acquirer_company,
            target_company=ev.target_company
        )

    @step
    async def identify_targets(self, ctx: Context, ev: TargetIdentificationEvent) -> FinancialAnalysisEvent:
        # Use web search to find information about the specific target company
        search_query = f"details about {ev.target_company} in {ev.industry} industry"
        result = self.target_id_agent.chat(
            f"Search for and provide detailed information about {ev.target_company} as an acquisition target in the '{ev.industry}' industry. Consider how it matches these criteria: '{ev.criteria}'. Use web search to find relevant details about the company. Provide a comprehensive profile of {ev.target_company} including its history, market position, financial highlights, and why it might be a suitable acquisition target for {ev.acquirer_company}."
        )

        # Store the analysis result
        result_str = str(result)
        self.all_analysis["target_identification"] = result_str

        # Set the target company in the context
        await ctx.set("target_company", ev.target_company)

        return FinancialAnalysisEvent(
            target_company=ev.target_company,
            acquirer_company=ev.acquirer_company
        )

    @step
    async def analyze_financials(self, ctx: Context, ev: FinancialAnalysisEvent) -> StrategicFitEvent:
        # Use web search to find financial information
        result = self.financial_agent.chat(
            f"Search for and analyze financial data for {ev.target_company} and {ev.acquirer_company}. Focus on revenue, profitability, growth rates, debt levels, and cash positions. Assess financial compatibility and health of both companies. Format your response as a structured financial analysis."
        )

        result_str = str(result)
        self.all_analysis["financial_analysis"] = result_str

        return StrategicFitEvent(
            financial_analysis=result_str,
            target_company=ev.target_company,
            acquirer_company=ev.acquirer_company
        )

    @step
    async def evaluate_strategic_fit(self, ctx: Context, ev: StrategicFitEvent) -> LegalRegulatoryEvent:
        # Use web search to gather information for strategic fit analysis
        result = self.strategic_agent.chat(
            f"Search for information about {ev.target_company} and {ev.acquirer_company} and evaluate their strategic fit. Consider their product portfolios, market positions, technologies, customer bases, and company cultures. Based on your findings and this financial analysis: {ev.financial_analysis}, provide a comprehensive strategic fit evaluation."
        )

        result_str = str(result)
        self.all_analysis["strategic_fit"] = result_str

        return LegalRegulatoryEvent(
            strategic_fit=result_str,
            target_company=ev.target_company,
            acquirer_company=ev.acquirer_company
        )

    @step
    async def assess_legal_risks(self, ctx: Context, ev: LegalRegulatoryEvent) -> SentimentEvent:
        # Use web search to assess legal and regulatory risks
        result = self.legal_agent.chat(
            f"Search for information about potential legal and regulatory issues for an acquisition of {ev.target_company} by {ev.acquirer_company}. Consider antitrust concerns, regulatory approval requirements, legal liabilities, compliance issues, and jurisdictional challenges. Based on your findings and this strategic fit analysis: {ev.strategic_fit}, provide a comprehensive legal risk assessment."
        )

        result_str = str(result)
        self.all_analysis["legal_risks"] = result_str

        return SentimentEvent(
            legal_analysis=result_str,
            target_company=ev.target_company,
            acquirer_company=ev.acquirer_company
        )

    @step
    async def analyze_sentiment(self, ctx: Context, ev: SentimentEvent) -> SynergyEvent:
        # Use web search to analyze market sentiment
        result = self.sentiment_agent.chat(
            f"Search for recent news, analyst reports, and social media sentiment regarding {ev.target_company} and {ev.acquirer_company}. Analyze market sentiment, media coverage, analyst opinions, and stakeholder sentiments regarding a potential acquisition. Provide a comprehensive sentiment analysis."
        )

        result_str = str(result)
        self.all_analysis["sentiment"] = result_str

        return SynergyEvent(
            sentiment_analysis=result_str,
            target_company=ev.target_company,
            acquirer_company=ev.acquirer_company
        )

    @step
    async def estimate_synergies(self, ctx: Context, ev: SynergyEvent) -> DealStructuringEvent:
        # Use web search to gather information for synergy estimation
        result = self.synergy_agent.chat(
            f"Based on information about {ev.target_company} and {ev.acquirer_company}, estimate potential synergies from this acquisition. Consider revenue synergies (cross-selling, market expansion), cost synergies (operational efficiencies, redundancy elimination), financial synergies (tax benefits, capital allocation), and strategic synergies (technology integration, talent acquisition). Include potential values and implementation timeline. Also consider this sentiment analysis: {ev.sentiment_analysis}"
        )

        result_str = str(result)
        self.all_analysis["synergies"] = result_str

        return DealStructuringEvent(
            synergy_estimation=result_str,
            target_company=ev.target_company,
            acquirer_company=ev.acquirer_company
        )

    @step
    async def structure_deal(self, ctx: Context, ev: DealStructuringEvent) -> ValuationEvent:
        # Use web search to gather information for deal structuring
        result = self.deal_agent.chat(
            f"Based on information about {ev.target_company} and {ev.acquirer_company}, recommend an optimal deal structure for this acquisition. Consider transaction type (merger, stock acquisition, asset purchase), consideration structure (cash, stock, mixed), governance arrangements, key terms, and timing considerations. Also consider this synergy estimation: {ev.synergy_estimation}"
        )

        result_str = str(result)
        self.all_analysis["deal_structure"] = result_str

        return ValuationEvent(
            deal_structure=result_str,
            target_company=ev.target_company,
            acquirer_company=ev.acquirer_company
        )

    @step
    async def perform_valuation(self, ctx: Context, ev: ValuationEvent) -> ReportGenerationEvent:
        # Use web search to gather information for valuation
        result = self.valuation_agent.chat(
            f"Based on information about {ev.target_company} and {ev.acquirer_company}, perform a comprehensive valuation analysis. Apply multiple valuation methods (DCF, comparable companies, precedent transactions), conduct premium analysis, review comparable transactions, and recommend a price range. Also consider this deal structure analysis: {ev.deal_structure}"
        )

        result_str = str(result)
        self.all_analysis["valuation"] = result_str

        return ReportGenerationEvent(
            valuation=result_str,
            target_company=ev.target_company,
            acquirer_company=ev.acquirer_company,
            all_analysis=self.all_analysis
        )

    @step
    async def generate_report(self, ctx: Context, ev: ReportGenerationEvent) -> ReportReviewEvent:
        # Compile all analyses into a comprehensive report
        analyses_text = "\n\n".join([
            f"--- {section.upper()} ---\n{content}"
            for section, content in ev.all_analysis.items()
        ])

        result = self.report_agent.chat(
            f"""Compile a comprehensive M&A analysis report for the acquisition of {ev.target_company} by {ev.acquirer_company} using all the following analyses:

            {analyses_text}

            Create a professional report with executive summary, key findings from all analyses, SWOT analysis, recommendations, and next steps."""
        )

        report = str(result)

        return ReportReviewEvent(report=report)

    @step
    async def review_report(self, ctx: Context, ev: ReportReviewEvent) -> StopEvent:
        # Create a weighted decision matrix and final recommendation
        result = self.review_agent.chat(
            f"""Review this M&A analysis report and create a weighted decision matrix:

            {ev.report}

            Evaluate key factors such as:
            1. Strategic value (25%)
            2. Financial impact (25%)
            3. Implementation feasibility (20%)
            4. Risk factors (20%)
            5. Timeline considerations (10%)

            Provide a final recommendation with a numerical score (0-100)."""
        )

        review = str(result)
        ctx.write_event_to_stream(ReviewResults(review=review))

        # Return both the report and review as the final result
        final_output = f"""
        # M&A ANALYSIS FINAL RESULTS

        ## COMPREHENSIVE REPORT
        {ev.report}

        ## DECISION MATRIX AND RECOMMENDATION
        {review}
        """

        return StopEvent(result=final_output)

# Example usage
import nest_asyncio
nest_asyncio.apply()

# Initialize the workflow
mna_workflow = MnAAnalysisWorkflow(timeout=300, verbose=True)  # Extended timeout for web searches

# Run the workflow
handler = mna_workflow.run(
    criteria="Luxury fashion brand with strong global recognition, annual revenue > $500M, profitable, positioned in the high-end/luxury market segment, and with complementary product lines or geographic market reach",
    industry="Fashion",
    acquirer_company="Prada",
    target_company="Versace",
    target_id_agent=target_identification_agent,
    financial_agent=financial_analysis_agent,
    strategic_agent=strategic_fit_agent,
    legal_agent=legal_risk_agent,
    sentiment_agent=sentiment_agent,
    synergy_agent=synergy_agent,
    deal_agent=deal_structure_agent,
    valuation_agent=valuation_agent,
    report_agent=report_agent,
    review_agent=review_agent
)

# Stream and display events
async for ev in handler.stream_events():
    if isinstance(ev, TargetIdentificationEvent):
        print(f"🔍 Identifying target: {ev.target_company} based on criteria: {ev.criteria} in {ev.industry}")
    elif isinstance(ev, FinancialAnalysisEvent):
        print(f"💰 Analyzing financials for {ev.target_company} and {ev.acquirer_company}")
    elif isinstance(ev, StrategicFitEvent):
        print(f"🧩 Evaluating strategic fit between {ev.target_company} and {ev.acquirer_company}")
    elif isinstance(ev, LegalRegulatoryEvent):
        print(f"⚖️ Assessing legal and regulatory risks")
    elif isinstance(ev, SentimentEvent):
        print(f"📊 Analyzing market sentiment")
    elif isinstance(ev, SynergyEvent):
        print(f"🔄 Estimating potential synergies")
    elif isinstance(ev, DealStructuringEvent):
        print(f"🏗️ Recommending deal structure")
    elif isinstance(ev, ValuationEvent):
        print(f"💵 Performing valuation analysis")
    elif isinstance(ev, ReportGenerationEvent):
        print(f"📝 Generating comprehensive M&A report")
    elif isinstance(ev, ReportReviewEvent):
        print(f"🔍 Reviewing final report")
    elif isinstance(ev, ReviewResults):
        print("\n==== DECISION MATRIX AND RECOMMENDATION ====")
        print(ev.review)

# Get final result
final_result = await handler
print("\n==== COMPLETE M&A ANALYSIS REPORT ====")
print(final_result)

Running step setup
Step setup produced event TargetIdentificationEvent
Running step identify_targets
Step identify_targets produced event FinancialAnalysisEvent
Running step analyze_financials
Step analyze_financials produced event StrategicFitEvent
Running step evaluate_strategic_fit
Step evaluate_strategic_fit produced event LegalRegulatoryEvent
Running step assess_legal_risks
Step assess_legal_risks produced event SentimentEvent
Running step analyze_sentiment
Step analyze_sentiment produced event SynergyEvent
Running step estimate_synergies
Step estimate_synergies produced event DealStructuringEvent
Running step structure_deal
Step structure_deal produced event ValuationEvent
Running step perform_valuation
Step perform_valuation produced event ReportGenerationEvent
Running step generate_report
Step generate_report produced event ReportReviewEvent
Running step review_report
Step review_report produced event StopEvent

==== DECISION MATRIX AND RECOMMENDATION ====
### Weighted Decision

## Result

In [10]:
print(final_result)


        # M&A ANALYSIS FINAL RESULTS

        ## COMPREHENSIVE REPORT
        # M&A Analysis Report: Acquisition of Versace by Prada

## Executive Summary
This report provides a comprehensive analysis of the proposed acquisition of Versace by Prada for approximately $1.4 billion. The acquisition is strategically significant, allowing Prada to enhance its luxury portfolio, expand market reach, and leverage operational synergies. Despite Versace's current financial challenges, the acquisition presents opportunities for growth and revitalization under Prada's stewardship. This report synthesizes key findings from target identification, financial analysis, strategic fit evaluation, legal risk assessment, sentiment analysis, synergy estimation, deal structure recommendations, and valuation analysis.

## Key Findings from All Analyses

1. **Target Identification**: Versace is a globally recognized luxury brand with a strong cultural presence but is currently facing declining revenues and pr

## Workflow Visualization

To visualize th workflow open this workflow.html file

In [11]:
from llama_index.utils.workflow import draw_all_possible_flows

draw_all_possible_flows(mna_workflow, filename="workflow.html")

workflow.html
